# **0. 참고자료**  
## **0-1. 도서**

## **0-2. 논문, 학술지**
## **0-3. 웹 사이트**
- 꿈 많은 사람의 이야기 : 파이썬과 함께 추천 시스템 이해하기 기본편 - content based filtering | [[블로그 링크]](https://lsjsj92.tistory.com/565?category=1066953) 
- Today I Learned. : 추천 시스템 - 컨텐츠 기반 필터링이란? (CBF) | [[블로그 링크]](https://western-sky.tistory.com/43)
- 비둘기 둥지 : [ML / 추천 시스템] 1. 추천 시스템에 대해 알아봅시다. | [[블로그 링크]](https://dove-nest.tistory.com/29)

## **0-4. 데이터셋 출처**
- Kaggle : Games of All Time from Metacritic | [[데이터 셋 링크]](https://www.kaggle.com/datasets/xcherry/games-of-all-time-from-metacritic)

## **0-5. 이미지 출처**

# **1. 컨텐츠 기반 필터링 (Contents Based Filtering)**
## **1-1. 컨텐츠 기반 필터링 이란**   
- 컨텐츠 기반 필터링(이하, CBF)는 사용자가 과거에 좋아했던 컨텐츠를 파악하고 비슷한 아이템을 추천해 주는 방식이다.  
    e.g.) 음식을 예로 들면 어떤 유저가 배달 어플을 통해 'KFC'와 '맥도날드'에 좋은 평점을 주었다면,  
          같은 햄버거 프랜차이즈인 '롯데리아'를 추천해 줄 수 있다.  
- 유저가 좋아한 아이템을 뽑아낸 목록을 Item Profile이라 하고,  
  Item Profile로부터 공통된 특징을 뽑아낸 결과를 User Profile이라고 한다.  
  e.g.) Item Profile | 'KFC', '맥도날드', '버거킹', ...  
          User Profile | '햄버거', '프랜차이즈', '패스트푸드', ...
          
## **1-2. CBF의 장점**

1. 다른 유저의 데이터를 필요로 하지 않는다.
2. 개인의 취향을 고려한 추천이 가능하다.
3. 새로운 아이템이나 대중적이지 않은 아이템도 추천할 수 있다.
4. 사용자에게 추천하는 이유에 대해 설명할 수 있다.
   → User Profile을 뽑아낼 때 구성한 Feature들로 설명가능하다.

⚠️ 1,2,3번으로 인해 얻을 수 있는 장점은 다음과 같다.  
  - 새로운 아이템이 추가되었을 때, 평가한 사함이 없어 추천이 어려운 Cold Start Problem에서 자유로워진다.  
  - 개인의 평가에 기반하기 때문에 모든 유저들이 모든 아이템에 대해 평가하지 않는 Sparsity Problem에서  
    자유로워 진다.
  - 아무도 평가하지 않은 새로운 아이템과 인기 없는 아이템도 해당 아이템들의 Feature들만 뽑아낼 수 있다면,  
    협업 필터링 구현시 발생하는 No first-rater Problem에서 자유로워진다.
    
## **1-3. CBF의 단점**

1. Feature를 뽑아내기 어려운 데이터들이 존재한다.  
   → 음악 같은 경우 장르가 너무 다양하기 때문에 공통된 Feature를 찾기 어렵다.
2. 처음 유입된 유저에 대해 추천할 수 없다.  
   → 주변 유저 평가는 영향을 받지 않지만, 아예 처음 유입된 유저는 그에 대한 데이터가 존재하지 않아 추천할 수 없다.
   
# **2. 실제 데이터 셋을 이용해 구현해보자**
- 평소에 게임을 좋아해서 캐글에 있는 Metacritic 리뷰 데이터 셋을 가져왔다.  
⚠️ Kaggle API를 이용해 데이터를 다운로드 받는 방법은 다음과 같다.
1. kaggle에서 API를 발급 받는다.
2. pip install kaggle을 통해 kaggle api 패키지를 설치한다.
3. 홈 디렉토리에 숨김폴더로 kaggle폴더를 만들고 kaggle API키를 복사한다.
4. kaggle API의 권한을 변경한다.

In [1]:
## kaggle API 사용을 위한 패키지 설치
!pip install kaggle

## json 파일 업로드 확인
!ls -1ha ~/project/TIL/api/kaggle.json

## 캐글 숨김 폴더 생성
!mkdir -p ~/.kaggle

## 토큰 파일 옮기고, 권한 부여
!cp  ~/project/TIL/api/kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

/home/jovyan/project/TIL/api/kaggle.json


In [2]:
## 데이터 셋 다운로드 받기
!kaggle datasets download -d xcherry/games-of-all-time-from-metacritic

## 파일 옮기고 압축풀기
!mkdir -p ~/project/TIL/AI_study/dataset/metacritic
!mv games-of-all-time-from-metacritic.zip ~/project/TIL/AI_study/dataset/zips/games-of-all-time-from-metacritic.zip
!unzip ~/project/TIL/AI_study/dataset/zips/games-of-all-time-from-metacritic.zip -d ~/project/TIL/AI_study/dataset/metacritic

 83%|███████████████████████████████▍      | 2.00M/2.42M [00:00<00:00, 11.3MB/s]
100%|██████████████████████████████████████| 2.42M/2.42M [00:00<00:00, 11.5MB/s]
Archive:  /home/jovyan/project/TIL/AI_study/dataset/zips/games-of-all-time-from-metacritic.zip
replace /home/jovyan/project/TIL/AI_study/dataset/metacritic/games_of_all_time.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: ^C


In [1]:
import pandas as pd
import numpy as np
import swifter
import ast

## **2-1. 데이터를 확인해보고, EDA 비스무리 하게 해보자**
- csv 포맷으로 되어있는 데이터 셋을 불러온다.
- 데이터 셋의 칼럼은 총 9개로 구성되어 있었으며, 아래처럼 구성되어 있었다.
|column|설명|
|:--|:--|
|game_name|게임 이름|
|meta_score|메타 크리틱 점수|
|user_score|유저 점수|
|platform|게임 플랫폼|
|description|게임에 관한 내용|
|url|메타크리틱 페이지 내 게임 링크|
|developer|개발사|
|genre|게임 장르|
|type|싱글 게임인지 멀티 게임인지|
|rating|게임 등급|
- 이 중에서 게임 장르를 User profile로 사용하여 게임을 추천하도록 하였다.

In [2]:
DATASET_PATH = '~/project/TIL/AI_study/dataset/metacritic'

csv = pd.read_csv(f'{DATASET_PATH}/games_of_all_time.csv')
csv = csv.iloc[:, :-1]
csv.head(3)

,game_name,meta_score,user_score,platform,description,url,developer,genre,type
0,The Legend of Zelda: Ocarina of Time,99.0,91.0,['nintendo-64'],"As a young boy, Link is tricked by Ganondorf, ...",https://www.metacritic.com/game/nintendo-64/th...,Nintendo,"['Action Adventure', 'Fantasy']",singleplayer
1,Super Mario Galaxy,97.0,91.0,['wii'],[Metacritic's 2007 Wii Game of the Year] The u...,https://www.metacritic.com/game/wii/super-mari...,Nintendo,"['Action', 'Platformer', '3D']",singleplayer
2,Super Mario Galaxy 2,97.0,91.0,['wii'],"Super Mario Galaxy 2, the sequel to the galaxy...",https://www.metacritic.com/game/wii/super-mari...,Nintendo EAD Tokyo,"['Action', 'Platformer', '3D']",singleplayer


In [3]:
csv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8831 entries, 0 to 8830
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   game_name    8831 non-null   object 
 1   meta_score   8831 non-null   float64
 2   user_score   8831 non-null   float64
 3   platform     8831 non-null   object 
 4   description  8831 non-null   object 
 5   url          8831 non-null   object 
 6   developer    8821 non-null   object 
 7   genre        8827 non-null   object 
 8   type         6727 non-null   object 
dtypes: float64(2), object(7)
memory usage: 621.1+ KB


In [4]:
csv = csv[csv['genre'].notna()]
csv['developer'] = csv['developer'].fillna('unknown')
csv['type']      = csv['type'].fillna('unknown')
csv.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8827 entries, 0 to 8830
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   game_name    8827 non-null   object 
 1   meta_score   8827 non-null   float64
 2   user_score   8827 non-null   float64
 3   platform     8827 non-null   object 
 4   description  8827 non-null   object 
 5   url          8827 non-null   object 
 6   developer    8827 non-null   object 
 7   genre        8827 non-null   object 
 8   type         8827 non-null   object 
dtypes: float64(2), object(7)
memory usage: 689.6+ KB


In [5]:
csv['genre'] = csv['genre'].swifter.apply(lambda x: ' '.join(ast.literal_eval(x)))
csv['platform'] = csv['platform'].swifter.apply(lambda x: set(ast.literal_eval(x)))

Pandas Apply:   0%|          | 0/8827 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/8827 [00:00<?, ?it/s]

In [6]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [7]:
vectorizer = CountVectorizer(ngram_range = (1, 3))
genre_vecs = vectorizer.fit_transform(csv['genre'])
genre_sims = cosine_similarity(genre_vecs, genre_vecs).argsort()[:, ::-1]

In [8]:
def filtering(name, top = 15):
    
    game_idx = csv[csv['game_name'] == name].index.values
    sim_idx  = genre_sims[game_idx, :top].reshape(-1)
    
    sim_idx  = sim_idx[sim_idx != game_idx]
    result   = csv.iloc[sim_idx].sort_values(['meta_score', 'user_score'], ascending = [False, False])
    
    return result

In [9]:
csv[csv['game_name'] == 'Super Mario Odyssey']

,game_name,meta_score,user_score,platform,description,url,developer,genre,type
4,Super Mario Odyssey,97.0,89.0,{switch},New Evolution of Mario Sandbox-Style Gameplay....,https://www.metacritic.com/game/switch/super-m...,Nintendo,Action Platformer 3D,singleplayer


In [86]:
filtering('Super Mario Odyssey')

,game_name,meta_score,user_score,platform,description,url,developer,genre,type
7613,Skylanders Swap Force,83.5,58.0,"{xbox-360, playstation-3, playstation-4, wii-u}",Embark on an exciting new adventure with the S...,https://www.metacritic.com/game/wii-u/skylande...,Vicarious Visions,Action Platformer 3D,unknown
7618,Sly 3: Honor Among Thieves,83.0,89.0,{playstation-2},Sly 3: Honor Among Thieves entices a broad ran...,https://www.metacritic.com/game/playstation-2/...,Sucker Punch,Action Platformer 3D,multiplayer
7614,Skylanders: Spyro's Adventure,79.5,52.5,"{3ds, playstation-3}","The small purple dragon returns, bringing with...",https://www.metacritic.com/game/3ds/skylanders...,Vicarious Visions,Action Platformer 3D,unknown
7612,Skylanders Giants,74.0,61.0,"{xbox-360, playstation-3, 3ds, wii-u}",Skylanders Giants unveils 16 new characters in...,https://www.metacritic.com/game/xbox-360/skyla...,Toys for Bob,Action Platformer 3D,unknown
7645,Sonic Adventure 2 Battle,73.0,85.0,{gamecube},Sonic and company are back and better than eve...,https://www.metacritic.com/game/gamecube/sonic...,Sonic Team,Action Platformer 3D,multiplayer
7650,Sonic Generations,73.0,78.0,"{playstation-3, pc, 3ds}",Set across three defining eras from 20 years o...,https://www.metacritic.com/game/pc/sonic-gener...,Sega,Action Platformer 3D,singleplayer
6855,NiGHTS: Journey of Dreams,69.0,86.0,{wii},"As a fantasy jester called NiGHTS, experience ...",https://www.metacritic.com/game/wii/nights-jou...,Sega,Action Platformer 3D,multiplayer
7656,Sonic and the Secret Rings,69.0,62.0,{wii},The game opens as Sonic is mysteriously transp...,https://www.metacritic.com/game/wii/sonic-and-...,Sonic Team,Action Platformer 3D,singleplayer
7651,Sonic Heroes,68.5,76.0,"{xbox, playstation-2}","In Sonic's most exciting adventure yet, you ma...",https://www.metacritic.com/game/xbox/sonic-heroes,Sonic Team,Action Platformer 3D,multiplayer
8574,WALL-E,67.0,85.0,{playstation-2},"What if mankind had to leave Earth, and somebo...",https://www.metacritic.com/game/playstation-2/...,Asobo Studio,Action Platformer 3D,multiplayer


- 이 데이터 셋으로는 그냥 이대로 마칠 수 있었지만, 실제 서비스처럼 플랫폼과 개발사를 고를 수 있게 함수를 만들어 보았다.

In [79]:


def advanced_filtering(top=15, **query):
    df = filtering(query['name'])
    
    platform, p_include  = query['platform']
    developer, d_include = query['developer']
    
    df = df[df['type'] == query['type']]
    
    if (p_include == True) and (d_include == True): 
        df = df[df['platform'].apply(lambda x: platform in x)]
        df = df[df['developer'].apply(lambda x: developer in x)] 
        
    elif (p_include == False) and (d_include == True): 
        df = df[df['platform'].apply(lambda x: platform not in x)]
        df = df[df['developer'].apply(lambda x: developer in x)] 
        
    elif (p_include == True) and (d_include == False):
        df = df[df['platform'].apply(lambda x: platform in x)]
        df = df[df['developer'].apply(lambda x: developer not in x)]
        
    else:
        df = df[df['platform'].apply(lambda x: platform not in x)]
        df = df[df['developer'].apply(lambda x: developer not in x)]
    
    return df
    

In [88]:
query = {
        'name' : 'Super Mario Odyssey',
        'type' : 'unknown',
         'platform' : ['pc', False], 
        'developer' : ['Toys for Bob', False]}


advanced_filtering(top = 15, **query)

,game_name,meta_score,user_score,platform,description,url,developer,genre,type
7613,Skylanders Swap Force,83.5,58.0,"{xbox-360, playstation-3, playstation-4, wii-u}",Embark on an exciting new adventure with the S...,https://www.metacritic.com/game/wii-u/skylande...,Vicarious Visions,Action Platformer 3D,unknown
7614,Skylanders: Spyro's Adventure,79.5,52.5,"{3ds, playstation-3}","The small purple dragon returns, bringing with...",https://www.metacritic.com/game/3ds/skylanders...,Vicarious Visions,Action Platformer 3D,unknown
7615,Skylar & Plux: Adventure on Clover Island,59.5,63.5,"{playstation-4, xbox-one}",Skylar & Plux: Adventure on Clover Island sets...,https://www.metacritic.com/game/xbox-one/skyla...,Right Nice Games,Action Platformer 3D,unknown
7644,Sonic Adventure,49.0,74.5,"{xbox-360, playstation-3}",The Dreamcast classic now makes its way onto t...,https://www.metacritic.com/game/playstation-3/...,Sonic Team,Action Platformer 3D,unknown
4162,"Monsters, Inc. Scream Arena",39.0,59.0,{gamecube},The monsters of Monstropolis have learned the ...,https://www.metacritic.com/game/gamecube/monst...,Radical Entertainment,Action Platformer 3D,unknown
